In [1]:
#%pip install scikit-image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import cifar100

import os 

In [ ]:
# Import the dataset
def read_images(dir):
 
    dir = 'data/airplane_cabin/' + dir
    imgs_names = os.listdir(dir)

    imgs = []
    for img in imgs_names:
        imgs.append(plt.imread(dir +'/' + img))

    return imgs
maps_ = os.listdir('data/airplane_cabin')
img = [read_images(dir) for dir in maps_[:20]]

In [ ]:
# split test and train
x_test = []
x_train = []
for i in range(len(img)):
    length = len(img[i])
    test_numbers = np.random.randint(0,length,int(np.floor(length/10)))
    x_test = x_test + [img[i][j] for j in test_numbers]
    x_train = x_train + [img[i][j] for j in range(length) if j not in test_numbers]

fig, ax = plt.subplots(1,2)
ax[0].imshow(x_train[-1])
ax[1].imshow(x_test[-1])


In [ ]:
for i in range(len(x_train)-1):
    if x_train[i].shape != (256,256,3):
        print(i, x_train[i].shape)
        x_train.pop(i)

In [ ]:
# Preprocess the data
x_train = np.array([img.astype('float32') / 255 for img in x_train])
x_test = np.array([img.astype('float32') / 255 for img in x_test])
x_train_gray = np.dot(x_train, [0.2989, 0.5870, 0.1140])
x_test_gray = np.dot(x_test, [0.2989, 0.5870, 0.1140])
x_train_gray = np.reshape(x_train_gray, (len(x_train_gray), 256, 256, 1))
x_test_gray = np.reshape(x_test_gray, (len(x_test_gray), 256, 256, 1))

fig, ax = plt.subplots(1,2)
ax[0].imshow(x_train_gray[0], cmap='gray')
_ = ax[1].imshow(x_test_gray[0], cmap='gray')



In [ ]:
# CAE architecture
input_img = Input(shape=(256, 256, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', learning_rate = 0.01)

In [ ]:
autoencoder.summary()

In [38]:
# Train the CAE
history = autoencoder.fit(x_train_gray, x_train, epochs=5, batch_size=32, shuffle=True, validation_data=(x_test_gray, x_test))

37/57 [==================>...........] - ETA: 8:56 - loss: 0.0099

39/57 [===================>..........] - ETA: 7:56 - loss: 0.0100

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')




In [ ]:
# Color restoration on test images
restored_images = autoencoder.predict(x_test_gray)

In [ ]:
# Display original grayscale and restored colored images
n = 5  # Number of images to display   
plt.figure(figsize=(20, 10))
for i in range(n):
    j = int(np.random.randint(0,len(x_test_gray),1))
    # Original grayscale image
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test_gray[j].reshape(256, 256), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Restored colored image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(restored_images[j])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
